<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 23 : Recherche dans une table</h1>

Une fois qu'un ensemble de données est chargé dans une table, il devient possible d'exploiter ces données à l'aide des opérations de manipulation de tableaux. On va, par exemple, pouvoir extraire des données cibles, tester la présence de certaines données ou faire des statistiques. Ces opérations sont applélées des **requêtes**.

Reprenons l'exemple de la table d'[élèves](Donnees/eleves.csv) :

In [ ]:
import csv

with open("Donnees/eleves.csv") as fichier:
    table = list(csv.DictReader(fichier, delimiter = ','))
    
def valide(dico: dict) -> dict:
    """Renvoie un dictionnaire après avoir validé et converti
    les valeurs associées aux clés du dictionnaire passé en paramètre"""
    prenom = dico["prénom"]
    jour = int(dico["jour"])
    mois = int(dico["mois"])
    annee = int(dico["année"])
    projet = dico["projet"]
    assert mois >= 1 and mois <= 12, "mois invalide dans le fichier CSV"
    return {"prénom" : prenom, "jour" : jour, "mois" : mois, "année" : annee, "projet" : projet}

eleves = [valide(ligne) for ligne in table]
eleves[0]

## Recherche
Considérons la fonction suivante, qui renvoie `True` ou `False` selon la présence ou l'absence d'un élément `val` dans un tableau `tab` 

In [ ]:
def appartient(val, tab: list) -> bool:
    """Vérifie si l'élément val appartient au tableau tab"""
    for ligne in tab:
        if val == ligne:
            return True
    return False

Cette fonction pourrait être appliquée directement à une table de données.  
Toutefois cela implique que `val`soit de la forme d'une ligne complète, par exemple :
```python
 {'prénom': 'Donald', 'jour': 10, 'mois': 1, 'année': 1938, 'projet': 'tout sur les algorithmes'}
```

Ceci demande de savoir à l'avance, exactement, ce que contient une ligne, ce qui ne correspond pas à l'utilisation courante d'une telle table.  
On cherchera, en général, plutôt à savoir s'il y a, dans la table, une ligne pour laquelle le prénom est `Donald`, pour obtenir, ensuite, certaines informations associées.

### Recherche en fonction d'un attribut clé
Une fonction recherchant dans une table un élève désigné par son prénom et renvoyant `True` ou `False` selon sa présence ou son absence peut être obtenue par une légère modification

In [ ]:
def appartient(prenom: str, tab: list) -> bool:
    """Vérifie si le tableau tab contient un dictionnaire
    dont la valeur associée à la clé prénom vaut prenom"""
    for ligne in tab:
        if ligne["prénom"] == prenom:
            return True
    return False

In [ ]:
appartient("Grace", eleves)

In [ ]:
appartient("John", eleves)

### Récupération d'une donnée simple
On peut déduire d'autres fonctions qui, au lieu de seulement indiquer la présence ou l'absence d'un élève, renvoient certaines des informations associées.

In [ ]:
def projet_de(prenom: str, tab: list) -> dict:
    """Renvoie le dictionnaire dont la valeur associée à la clé prénom vaut prenom
    Renvoie None si le dictionnaire n'existe pas"""
    for ligne in tab:
        if ligne["prénom"] == prenom:
            return ligne["projet"]

Cette fonction permet de récupérer le projet d'un élève désigné par son prénom. La fonction renvoie `None` si aucun élève n'a le prénom `prenom`.

In [ ]:
projet_de("Alan", eleves)

Toutefois si deux élèves ont le même prénom, un seul projet est renvoyé (celui de l'élève dont la ligne arrive en premier).

On peut donc vouloir préciser la recherche, en tenant compte, par exemple, à la fois du prénom et de l'année de naissance.

In [ ]:
def projet_de(prenom: str, annee: int, tab: list) -> str:
    """Renvoie la valeur associée à la clé projet du 
    dictionnaire dont la valeur associée à la clé prénom vaut prenom
    et la valeur associée à la clé année vaut annee.
    Renvoie None si le dictionnaire n'existe pas"""
    for ligne in tab:
        if ligne["prénom"] == prenom and ligne["année"] == annee:
            return ligne["projet"]

In [ ]:
projet_de("Alan", 1922, eleves)

## Agrégation
Les opérations d'**agrégation** combinent les données de plusieurs lignes pour produire un résultat et, en particulier, une statistique sur ces données.  

On pourra, par exemple, compter le nombre de lignes répondant à une certaine condition, ou calculer la valeur moyenne d'un attribut.

### Comptage d'occurences
Les fonctions visant à compter le nombre d'occurrences dans un tableau peuvent, comme les fonctions de recherche, être adaptées pour compter, dans une table, le nombre de lignes validant une certaine condition.

In [ ]:
def eleves_nes_en(annee: int, tab: list) -> int:
    """Renvoie le nombre de dictionnaires pour lesquels 
    la valeur associée à la clé année vaut annee."""
    nb = 0
    for ligne in tab:
        if ligne["année"] == annee:
            nb += 1
    return nb

In [ ]:
eleves_nes_en(1912, eleves)

### Sommes et moyennes
De manière générale, les opérations d'agrégation peuvent être réalisées en utilisant des accumulateurs qui enregistrent progressivement un bilan du parcours de la table.  

Pour calculer l'âge moyen de notre classe, à la fin de l'année `an`, on peut utiliser un accumulateur pour les âges.

In [ ]:
def age_moyen(an: int, tab: list) -> float:
    """Renvoie la moyenne d'age de la classe à la fin de l'année an.
    Ne gère pas le cas d'une liste vide"""
    somme = 0
    for ligne in tab:
        somme += an - ligne["année"]
    return somme / len(tab)

In [ ]:
age_moyen(2021, eleves)

On peut adapter le code pour ne calculer la moyenne que d'une certaine catégorie d'élèves.  
Pour calculer la moyenne d'âge des élèves portant un certain prénom `prenom`, on peut utiliser la fonction suivante :

In [ ]:
def age_moyen_de(prenom: str, an: int, tab: list) -> float:
    """Renvoie la moyenne d'age des élèves de la classe 
    dont la valeur associée à la clé prénom est prenom, à la fin de l'année an.
    Ne gère pas le cas ou un aucun élève ne s'appelle prenom."""
    somme = 0
    nb = 0
    for ligne in tab:
        if ligne["prénom"] == prenom:
            somme += an - ligne["année"]
            nb += 1
    return somme / nb

In [ ]:
age_moyen_de("Alan", 2021, eleves)

## Sélection de lignes
Les opérations présentées précédemment analysent la table pour produire un résultat simple, sous la forme d'une unique valeur (résultat d'un test, valeur d'un attribut, valeur agrégée,  etc ...).

Une autre opération courante, appelée **sélection**, consiste à produire une nouvelle table en extrayant, de la table d'origine, toutes les lignes vérifiant une certaine condition.  
On pourra, par exemple, sélectionner l'ensemble des lignes valides, ou l'ensemble des lignes relatives à des élèves nés avant 1940.

Pour réaliser ce genre d'opération, on peut réutiliser la technique de construction d'un tableau en compréhension.

In [ ]:
t = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
t1 = [i for i in tab if i % 2 == 1]
print(t1)

Ce même principe peut être appliqué à une table de données, c'est-à-dire à un tableau de dictionnaires, en faisant porter la condition sur les valeurs de certains attributs.

In [ ]:
# Sélection des lignes concernant les élèves nés en janvier ou février
sel1 = [ligne for ligne in eleves if ligne["mois"] <= 2]
print(sel1)

On parle de **sélection** des lignes vérifiant cette condition, ou encore de **filtrage**.

La condition introduite par le `if` peut être une condition arbitraire. En particulier, il est possible de combiner les conditions sur les valeurs de différents attributs.

In [ ]:
# Sélection des lignes concernant les élèves nés avant le 15 février
sel2 = [ligne for ligne in eleves if ligne["mois"] == 1 \
 or (ligne["mois"] == 2 and ligne["jour"] < 15)]
print(sel2)

On peut déléguer la condition à une fonction :

In [ ]:
def signe_sagittaire(ligne: dict) -> bool:
    """Verifie si la ligne correspond à un élève né avant le 15 février."""
    return (ligne["mois"] == 11 and ligne["jour"] >= 23) \
           or (ligne["mois"] == 12 and ligne["jour"] <= 21)

sel3 = [eleve for eleve in eleves if signe_sagittaire(eleve)]
print(sel3)

### Sélection ligne à ligne
Une manière alternative de sélectionner des lignes consiste à créer d'abord un tableau vide auquel on ajoute une à une les lignes sélectionnées.
```python
tab = []
for ligne in table:
    if signe_sagittaire(ligne):
        tab.append(ligne)
```

La variable `tab` désigne un tableau nouvellement créé contenant, dans l'ordre, les lignes de la table `eleves` présentant des dates valides.

La méthode [`append`](https://docs.python.org/fr/3/tutorial/datastructures.html#more-on-lists) modifie le tableau auquel elle est appliquée.

### Application : sélection des lignes valides
Dans un fichier tel que `eleves.csv` la colonne `jour` n'est censée contenir que des nombres compris entre `1` et `31`, voire entre `1` et `28`, `29` ou `30` en fonction du mois et de l'année.  
Cependant, rien ne garantit que ce soit toujours le cas lorsque l'on charge le fichier, qui peut être corrompu ou invalide pour de multiples raisons.  
Plutôt que d'arrêter le programme à la première ligne invalide rencontrée, nous pouvons utiliser une opération de sélection pour construire une table contenant chaque ligne valide du fichier de données et ignorant les autres.

In [ ]:
import csv

with open("Donnees/eleves.csv") as fichier:
    eleves = list(csv.DictReader(fichier, delimiter = ','))

In [ ]:
sel4 = [ligne for ligne in eleves if 1 <= int(ligne["jour"]) <= 31 \
                           and 1<= int(ligne["mois"]) <= 12 \
                           and int(ligne["année"]) <= 2019]
print(sel4)

Rappelons que la table directement obtenue par la lecture du fichier CSV ne contient, dans un premier temps, que des chaînes de caractères.  

Pour éviter que le programme soit interrompu si la valeur passée en argument de `int` n'est pas un nombre, on peut utiliser la méthode [`isdecimal()`](https://docs.python.org/fr/3/library/stdtypes.html?#str.isdecimal).

In [ ]:
# validation d'une table d'élèves

def est_valide(ligne: dict) -> bool:
    """Verifie si les valeurs associées aux clés du dictionnaire sont valides."""
    jour = ligne["jour"]
    mois = ligne["mois"]
    annee = ligne["année"]
    return jour.isdecimal() and 1 <= int(jour) <= 31 \
       and mois.isdecimal() and 1 <= int(mois) <= 12 \
       and annee.isdecimal() and int(annee) <= 2019

def conversion(ligne: dict) -> dict:
    """Convertit les valeurs associées aux clés du dictionnaire."""
    return { "prénom" : ligne["prénom"], "jour" : int(ligne["jour"]),
             "mois" : int(ligne["mois"]), "année" : int(ligne["année"]),
             "projet" : ligne["projet"] }

eleves = [conversion(eleve) for eleve in eleves if est_valide(eleve)]
eleves[0]

## Sélection de lignes et de colonnes
La construction de tableau par compréhension peut effectuer un calcul.

In [ ]:
t = [i * i for i in range(7)]
print(t)

Cette possibilité peut être utilisée dans le cas particulier d'une table de données, par exemple pour sélectionner une colonne pariculière plutôt que l'intégralité de chaque ligne.

In [ ]:
sel5 = [ligne["prénom"] for ligne in eleves]
print(sel5)

On construit ainsi un tableau contenant les prénoms de chaque élève de la table `eleves`.  

On appelle cette opération une **projection**. La projection sur une colonne peut être combinée à la sélection d'un ensemble de lignes.  
Par exemple, pour obtenir les prénoms des élèves nés avant le 15 février, on peut exécuter l'instruction :

In [ ]:
sel6 = [ligne["prénom"] for ligne in eleves \
                 if ligne["mois"] == 1 \
                 or (ligne["mois"] == 2 and ligne["jour"] < 15)]
print(sel6)

L'opération de projection est notamment utilisée pour construire une nouvelle table contenant seulement une partie des colonnes de la table d'origine.  

Plutôt que d'extraire uniquement la valeur d'un attribut, on peut reconstruire un nouveau dictionnaire contenant les valeurs des attributs qui nous intéressent.

In [ ]:
tab1 = [{"prénom" : ligne["prénom"], "projet" : ligne["projet"]} for ligne in eleves]
print(tab1)

On construit ainsi, pour chaque ligne de la table `eleves` un $n$-uplet contenant un attribut `"prénom"` et un attribut `"projet"`, dont les valeurs sont exactement les valeurs des attributs  de même nom dans la ligne considérée.  
Autrement dit, on extrait de la table, une table ne conservant que les prénoms et les projets, on encore, on effectue une projection sur ces deux colonnes.

Plus généralement, on peut effectuer n'importe quelle opération sur les lignes au moment de faire une sélection ou une projection et notamment produire une table contenant de nouvelles colonnes.  

In [ ]:
tab2 = [{"prénom" : ligne["prénom"], "âge" : 2021 - ligne["année"]} for ligne in eleves]
print(tab2)

In [ ]:
tab3 = [{"prénom" : ligne["prénom"], "projet" : ligne["projet"]} 
 for ligne in eleves if ligne["mois"] == 1 \
 or ligne["mois"] == 2 and ligne["jour"] < 15]
print(tab3)

## Utilisation de la bibliothèque [pandas](https://pandas.pydata.org/)
Nous pouvons aussi utiliser la bibliothèque pandas qui permet d’exprimer de façon simple, lisible et concise
de genre d'interrogation.

### Lecture et importation du fichier CSV
On s'interesse au fichier [`countries.csv`](Donnees/countries.csv).

In [ ]:
import pandas

pays = pandas.read_csv("Donnees/countries.csv", delimiter = ';', keep_default_na = False)

### Interrogations simples


In [ ]:
# Noms des pays où l’on paye en euros
pays[pays.currency_code == 'EUR'].name

In [ ]:
# Codes des monnaies appelées Dollar
pays[pays.currency_name == 'Dollar'].currency_code.unique()

## Bases de données
Les opérations d'extraction sont du domaine de la manipulation de **bases de données**. 

* L’utilisation la plus courante consiste à lire des données issues de la base de données.   
Dans le langage SQL (Structured Query Language), cela s’effectue grâce à la commande `SELECT`, qui retourne des enregistrements dans un tableau de résultat.  
Cette commande peut **sélectionner** une ou plusieurs colonnes d’une table.  
L’utilisation basique de cette commande s’effectue de la manière suivante:

```sql
SELECT nom_du_champ 
FROM nom_table
```


* La commande `WHERE` dans une requête SQL permet d’**extraire** les lignes d’une base de données qui respectent une **condition**. Cela permet d’obtenir uniquement les informations désirées.
La commande `WHERE` s’utilise en complément à une requête utilisant `SELECT`.  
La façon la plus simple de l’utiliser est la suivante:

```sql
SELECT nom_colonnes 
FROM nom_table 
WHERE condition
```

## Exercices

### Exercice 1
Imaginons un bon de commande représenté dans une table de données dans laquelle chaque ligne correspond à un produit commandé et contient quatre attributs : 
* la référence du produit et sa désignation (deux chaînes de caractères)
* le prix unitaire (un nombre décimal)
* la quantité commandée (un nombre entier)

Voici un exemple

| réf   | désignation            | prix | qté |
|-------|------------------------|------|-----|
| 18653 | lot crayons HB         | 2,30 | 1   |
| 15223 | stylo rouge            | 1.50 | 3   |
| 20112 | cahier petits carreaux | 3.50 | 2   |

1. Ecrire une fonction `verifie_quantites` qui analyse un bon de commande et renvoie `True` si, pour chaque produit commandé, la quantité est bien positive.
2. Ecrire une fonction `nombre_produit` qui renvoie le nombre total de produits demandé dans un bon de commande donné en argument (en ne comptant que les quantités positives).

### Exercice 2
On se place dans le contexte de l'exercice précédent.
1. Ecrire une fonction `purge_commande` qui prend en paramètre un bon de commande `b` et renvoie un nouveau bon de commande dans lequel seuls les produits commandés en quantités strictement supérieures à 0 sont conservés.
2. Ecrire une fonction `prix` qui renvoie le prix total d'un bon de commande après l'avoir purgé.

### Exercice 3
On se place, encore, dans le contexte de l'exercice 1 et on se donne, en plus, un dictionnaire `poids_produits` dont les clés sont les numéros de référence de tous les produits du catalogue, et les valeurs associées sont les poids exprimés en grammes.
1. Ecrire une fonction `poids_commande` qui renvoie le poids total d'une commande, en supposant que les quantités sont bien toutes positives.
2. Ecrire une fonction `articles_lourds` qui renvoie un nouveau bon de commande dans lequel seuls les produits dont le poids unitaire dépasse 200 grammes.

### Exercice 4
On se place, encore, dans le contexte de l'exercice 1 et on se donne, en plus, un dictionnaire `tarifs` dont les clés sont les numéros de référence de tous les produits du catalogue, et les valeurs associées sont les prix unitaires.
1. Ecrire une fonction `verifie_commande` qui analyse un bon de commande et renvoie `True` si les tarifs indiqués sont les bons.
2. Ecrire une fonction `cherche_erreurs` qui analyse un bon de commande `b` et renvoie une nouvelle table contenant, pour chaque ligne de `b`erronée, la référence du produit, le prix indiqué dans le bon de commande et le prix du catalogue.

### Exercice 5
Rappelons qu'une table peut contenir des attributs indéfinis (avec la valeur `None`).
1. Ecrire une fonction `nb_indefinis`qui analyse une table et renvoie le nombre d'attributs de ses lignes valant `None`.
2. Ecrire une fonction `nb_lignes_incompletes` qui analyse une table et renvoie le nombre de ligne comportant, au moins, un attribut valant `None`.

### Exercice 6
On considère un registre de ventes d'appartements représenté par une table de données `registre` dont chaque ligne décrit un bien vendu avec quatre attributs : `lat` (latitude), `long` (longitude), `surface` (en $m^2$), `prix` (en €).  
Les deux premiers atributs ont pour valeur des nombres décimaux, et les deux derniers des nombres entiers.  

Voic un exemple :

| lat     | long   | surface |   prix  |
|---------|--------|:-------:|:-------:|
| 48,6938 | 6,1893 | 91      | 169 000 |
| 48,6907 | 6,1809 | 19      | 55 000  |
| 48,6955 | 6,1811 | 75      | 176 000 |

1. Ecrire une fonction `surface_sup(s, registre)` qui renvoie le nombre d'appartements vendus dont la surface est supérieure ou égale à `s`.
2. Ecrire une fonction `prix_inf(p, registre)` qui renvoie le nombre d'appartements vendus dont le prix est inférieur ou égal à `p`.
3. Ecrire une fonction `surface_sup_prix_inf(s, p, registre)` qui renvoie le nombre d'appartements vendus pour lesquels, à la fois la surface est inférieure ou égale à `s` et, le prix est inférieur ou égal à `p`.

### Exercice 7
Reprendre les questions de l'exercice précédent, en renvoyant, cette fois, les tables obtenues en sélectionnant les lignes répondant à chacune des propriétés.

### Exercice 8
On se place dans le contexte de l'exercice 6 et on suppose que le registre contient, au moins, les lignes de l'exemple.
1. Ecrire une fonction `prix_m2_max(registre)` qui calcule le prix par mètre carré le plus élevé.
2. Ecrire une fonction `prix_moyen(registre)` qui calcule le prix moyen des appartements vendus.
3. Ecrire un programme `prix_moyen_familial(registre)` qui calcule le prix moyen d'un appartement dont la surface est comprise entre 70 et 100 mètres carrés.

### Exercice 9
On se place dans le contexte de l'exercice 6.
1. Ecrire une fonction `plus_proche(x, y, registre)` qui renvoie la ligne du registre concernant l'appartement le plus proche du point défini par les coordonnées de latitude `x` et longitude `y`.  
On fera une approcimation, en considérant que latitude et longitude correspondent à des coordonnées dans le plan cartésien, et on utilisera la distance euclidienne habituelle
2. Ecrire une fonction `dans_un_rayon_de(r, x, y, registre)` qui renvoie une table formée par les lignes du registre concernant des appartements à une distance inférieure ou égale à `r` du point de coordonnées `x`et `y`.
3. Reprendre la question précédente, mais en renvoyant une table avec seulement trois colonnes, indiquant les coordonnées et le prix au mètre carré.

## Liens :
* Document accompagnement Eduscol : [Manipulation de tables](https://cache.media.eduscol.education.fr/file/NSI/78/1/RA_Lycee_G_NSI_trtd_tables_1170781.pdf)
* Document accompagnement Eduscol : [Manipulation de tables avec la bibliothèque Pandas](https://cache.media.eduscol.education.fr/file/NSI/78/0/RA_Lycee_G_NSI_trtd_pandas_1170780.pdf)
* CNAM : [Travaux pratiques Bases de données](https://deptfod.cnam.fr/bd/tp)
* W3School : [The Try-SQL Editor](https://www.w3schools.com/sql/trysql.asp?filename=trysql_op_in)